# HashiCorp Vault Demo for Transit Engine and Advanced Data Protection (ADP) features.

This demo shows how HashiCorp Vault Transit engine encryption and ADP features work.  ADP features include Format Preserving Encryption (FPE), Data Masking, and Tokenization.

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

Note: You will need Vault to be installed with an ADP Transform license add-on.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

In [ ]:
# Install openssl.  This is used for the RSA asymmetric encrypt/decrypt demo.
brew install openssl

# Setting up HashiCorp Vault

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# Vault Transit Engine
This section demonstrates the base encryption/decryption functions.

Ref:
- https://developer.hashicorp.com/vault/docs/secrets/transit
- https://developer.hashicorp.com/vault/api-docs/secret/transit

In [ ]:
# Enable transit engine at the default mount path
#vault secrets disable transit
vault secrets enable transit

In [ ]:
# Create a key
vault write -f transit/keys/demo-key

# Policy sample to use key
# Ref: https://developer.hashicorp.com/vault/tutorials/encryption-as-a-service/eaas-transit?variants=vault-deploy%3Aselfhosted#create-a-token-for-vault-clients

In [ ]:
# Encrypt plain text with key and store cipher text
# Before encrypting, value will be base64 encoded
vault write -f transit/encrypt/demo-key plaintext=$(base64 <<< "4111 1111 1111 1111")

# Store cipher text
export CIPHERTEXT=$(vault write -f -field=ciphertext transit/encrypt/demo-key plaintext=$(base64 <<< "4111 1111 1111 1111"))
echo
echo "Encrypted value: $CIPHERTEXT"

In [ ]:
# Decrypt cipher text
echo "Encrypted value: $CIPHERTEXT"
echo
echo "Decrypted value: $(base64 --decode <<< $(vault write -field=plaintext transit/decrypt/demo-key ciphertext=$CIPHERTEXT))"


In [ ]:
# Demo key rotation
vault write -f transit/keys/demo-key/rotate

In [ ]:
# Show that the key_version is now incremented
vault write -f transit/encrypt/demo-key plaintext=$(base64 <<< "4111 1111 1111 1111")

In [ ]:
# Note the following commands should show errors and is expected

# Show that you cannot export a key if it is not marked exportable
vault read transit/export/encryption-key/demo-key

# Show that you cannot backup the key
vault read transit/backup/demo-key

# Show that you cannot delete a key if it is not marked as deletion allowed
vault delete transit/keys/demo-key

In [ ]:
# Let's create a new key to show export and deletion capability

# Mark as exportable on creation
vault write -f transit/keys/demo-key2 exportable=true

In [ ]:
# Configure allow deletion and allows backup/restore in plaintext format
vault write transit/keys/demo-key2/config deletion_allowed=true allow_plaintext_backup=true

In [ ]:
# Show that you can now export the key
vault read transit/export/encryption-key/demo-key2
echo
# Show that you can do a backup of the key
vault read transit/backup/demo-key2
export KEY_BACKUP=$(vault read -field=backup transit/backup/demo-key2)

In [ ]:
# Show that you can delete the key now
vault delete transit/keys/demo-key2

# Show that the key is deleted
vault read transit/export/encryption-key/demo-key2

In [ ]:
# Let's restore the key now
echo "Key Backup: $KEY_BACKUP"
# Restore the key
vault write transit/restore/demo-key2 backup=$KEY_BACKUP
echo
# Show that the key is restored
vault read transit/export/encryption-key/demo-key2

## Vault Transit Engine - Asymmetric Encryption using RSA key

This optional demo shows how an external user can use the public key portion of an transit engine RSA key to encrypt a piece of sensitive data.  The encrypted data can then be sent to the key owner for decryption using the private key stored in Vault.

Note: RSA is only able to encrypt data to a maximum amount equal to your key size (2048 bits = 256 bytes), minus any padding and header data (11 bytes for PKCS#1 v1.5 padding, PSS is size of hash + salt, 42 bytes for OAEP padding).

In [ ]:
# Note: Please enable the transit engine before executing this.

# Create a new RSA 4096 bit key
vault write transit/keys/demo-key-rsa type=rsa-4096

In [ ]:
# View the Transit RSA key information in JSON.  Note the public key portion is under .data.keys."1".public_key
vault read -format=json transit/keys/demo-key-rsa

In [ ]:
# Export the Public Key portion of the RSA key
vault read -format=json transit/keys/demo-key-rsa | jq -r .data.keys.\"1\".public_key > demo-key-rsa.pub

# Show the public key value
cat demo-key-rsa.pub

In [ ]:
# The public key is then passed to an external user to encrypt the user's secret.

# Create a secret and encrypt it using openssl
echo "This is a top secret message" > secret.txt

openssl pkeyutl -encrypt -in secret.txt -out secret.enc -pubin -inkey demo-key-rsa.pub -pkeyopt rsa_padding_mode:oaep -pkeyopt rsa_oaep_md:sha256 

# Show that the secret is now encrypted
echo "Encrypted Value: $(cat secret.enc)"

In [ ]:
# The encrypted file is now sent to the private key owner

# Key owner now uses the Vault to decrypt the contents using the initial key version (v1)
# Decrypt operation - show the json structure
vault write -format=json transit/decrypt/demo-key-rsa ciphertext=vault:v1:$(base64 -i secret.enc)

In [ ]:
# Decrypt and base64 decode to get the secret value
echo "Secret Value: $(vault write -format=json transit/decrypt/demo-key-rsa ciphertext=vault:v1:$(base64 -i secret.enc) | jq -r .data.plaintext | base64 -d)"

## Vault Transit Engine - Signing and Verification with Vault using RSA key

This optional demo show how we can use Vault to sign and verify files.

In [ ]:
# Create a message for signing
echo "This is a my message for signing verification" > message.txt

In [ ]:
# Note: Please enable the transit engine before executing this.

# Create an RSA key in Vault for signing
# Note we are marking it as exportable so that we can later show offline verification use cases
vault write transit/keys/demo-sign-key-rsa type=rsa-4096 exportable=true

In [ ]:
# Export the public and private portions of the RSA key for use later in the offline signing/verification scenarios
vault read -format=json transit/export/encryption-key/demo-sign-key-rsa | jq -r '.data.keys."1"' > demo-sign-key-rsa-private.pem
vault read -format=json transit/keys/demo-sign-key-rsa | jq -r .data.keys.\"1\".public_key > demo-sign-key-rsa-public.pem

### Online signing and verification

This shows how Vault is used for both signing and verification.

In [ ]:
# We need to base64 encode the message before signing it with Vault
export B64_MESSAGE=$(base64 -i message.txt)

echo "Message (Base64): $B64_MESSAGE"

In [ ]:
# We will be using PSS padding for our signing.  pkcs1v15 and pss is supported.
# Ref: https://developer.hashicorp.com/vault/api-docs/secret/transit#signature_algorithm
# Note that pkcs1v15 is deterministic.  i.e. Same signature is produced for the same message and key.
# PSS is randomized unless you use a zero-length salt.  i.e. Different signature is produced each time.

# Sign the message and get the signature
export B64_SIGNATURE=$(vault write -format=json transit/sign/demo-sign-key-rsa/sha2-256 \
    input="$B64_MESSAGE" \
    signature_algorithm="pss" | \
    jq -r '.data.signature')

echo "Signature (Base64): $B64_SIGNATURE"

In [ ]:
# Message and signature is sent to destination user for verification.
# Verify the signature.  You should see valid=true.
vault write transit/verify/demo-sign-key-rsa/sha2-256 \
    signature_algorithm="pss" \
    input=$B64_MESSAGE \
    signature=$B64_SIGNATURE

### Online signing and Offline verification

This shows how we can do offline verification using OpenSSL.  Earlier we have already signed the message using vault.

In [ ]:
# Remove "vault:v1:" from signature
export RAW_B64_SIGNATURE="$(cut -d':' -f3 <<< $B64_SIGNATURE)"
echo "Raw Signature (base64): $RAW_B64_SIGNATURE"
echo
# Save signature as a binary file (i.e. base64 decode)
echo $RAW_B64_SIGNATURE | base64 --decode > message.signature
echo "Raw Signature: "
cat message.signature


In [ ]:
# We send the message file, signature file, and public key file to the destination user.
# Destination user can verify the message offline using OpenSSL.  You should see a "Verified OK" message.
# This verifies that the message contents have not been touched.
openssl dgst -sha256 -verify demo-sign-key-rsa-public.pem -sigopt rsa_padding_mode:pss -signature message.signature message.txt

### Offline signing and Online verification

This shows how we can do offline signing using OpenSSL and a private key created in Vault.  Vault can then be used to verify the signature online.

In [ ]:
# Sign the message using OpenSSL and save the signature file
openssl dgst -sha256 -sigopt rsa_padding_mode:pss -sign demo-sign-key-rsa-private.pem -out message.signature message.txt

echo "Raw Signature (from OpenSSL):"
cat message.signature

In [ ]:
# If you have the public key portion of the RSA key, you can also use OpenSSL to verify it locally.
# You should see a "Verified OK" message.
openssl dgst -sha256 -verify demo-sign-key-rsa-public.pem -sigopt rsa_padding_mode:pss -signature message.signature message.txt

In [ ]:
# We send the message file, signature file to the destination user.
# Destination user can verify the message online using Vault.  You should see valid=true.
# This verifies that the message contents have not been touched.
export B64_MESSAGE=$(base64 -i message.txt)
export B64_SIGNATURE=vault:v1:$(base64 -i message.signature)
echo "Message (Base64): $B64_MESSAGE"
echo "Signature (Base64): $B64_SIGNATURE"
echo
vault write transit/verify/demo-sign-key-rsa/sha2-256 \
    signature_algorithm="pss" \
    input=$B64_MESSAGE \
    signature=$B64_SIGNATURE

## Setup Vault's Transform Engine (ADP Feature)

In [ ]:
# Enable transform engine at the default mount path
# Policy for configuration - https://developer.hashicorp.com/vault/tutorials/adp/transform#policy-requirements
vault secrets enable transform

# Format Preserving Encryption (FPE) Demo - Credit Card Number

In [ ]:
# Create a role named "payments" with "card-number" transformation attached
vault write transform/role/payments transformations=card-number

In [ ]:
# Verify the role "payments" is created
vault list transform/role

In [ ]:
# View existing templates.  Show that we will be using "builtin/creditcardnumber"
vault list transform/template

In [ ]:
# Create a transformation named "card-number" which will be used by role "payments" to transform credit card numbers.
# This uses the built-in template builtin/creditcardnumber to perform format-preserving encryption (FPE). 
vault write transform/transformations/fpe/card-number \
    template="builtin/creditcardnumber" \
    tweak_source=internal \
    allowed_roles=payments

In [ ]:
# Verify the "card-number" transformation is created
vault list transform/transformations/fpe

In [ ]:
# View the "card-number" transformation details
# https://developer.hashicorp.com/vault/tutorials/adp/transform#policy-requirements
vault read transform/transformations/fpe/card-number

In [ ]:
# Encode credit card number using FPE
vault write transform/encode/payments value=1111-2222-3333-4444 transformation=card-number

# Store encoded value
export FPE_ENCODED_VALUE=$(vault write -format json transform/encode/payments value=1111-2222-3333-4444 \
  transformation=card-number | jq -r .data.encoded_value)

In [ ]:
# Decode credit card number
vault write transform/decode/payments value=$FPE_ENCODED_VALUE transformation=card-number

# FPE Demo - NRIC

In [ ]:
# Create template for Singapore NRIC.  Note that we are not changing the front and ending letters.  
vault write transform/template/sg-nric-tmpl \
    type=regex \
    pattern="[stfgSTFG](\d{7})[a-zA-Z]" \
    alphabet=builtin/numeric 

In [ ]:
# View added template
vault list transform/template

In [ ]:
# Create transformation named "sg-nric" with the "sg-nric-tmpl" template
vault write transform/transformations/fpe/sg-nric \
    template=sg-nric-tmpl \
    tweak_source=internal \
    allowed_roles=hr-role


In [ ]:
# Create a role named "hr-role" with "sg-nric" transformation attached
vault write transform/role/hr-role transformations=sg-nric

In [ ]:
# Encode a NRIC value
vault write transform/encode/hr-role value="S7012345Z" transformation=sg-nric

export FPE_ENCODED_VALUE=$(vault write -format json transform/encode/hr-role value="S7012345Z" transformation=sg-nric | jq -r .data.encoded_value)

In [ ]:
# Decode NRIC
echo -e "FPE Encoded Value is: $FPE_ENCODED_VALUE\n"
vault write transform/decode/hr-role value=$FPE_ENCODED_VALUE transformation=sg-nric

# Data Masking Demo - IP Address

In [ ]:
# Create template for IP address.  I want to mask all digits.
vault write transform/template/ip-address-tmpl type=regex \
    pattern="(2(?:[0-4][0-9]|5[0-5])|[0-1]?[0-9]?[0-9])\.(2(?:[0-4][0-9]|5[0-5])|[0-1]?[0-9]?[0-9])\.(2(?:[0-4][0-9]|5[0-5])|[0-1]?[0-9]?[0-9])\.(2(?:[0-4][0-9]|5[0-5])|[0-1]?[0-9]?[0-9])" \
    alphabet=builtin/numeric

In [ ]:
# View added template
vault list transform/template

In [ ]:
# Create a transformation named "ip-address" which will be used by role "logging-role" 
vault write transform/transformations/masking/ip-address \
    template=ip-address-tmpl \
    masking_character=# \
    allowed_roles='logging-role'


In [ ]:
# Create a role named "logging-role" with "ip-address" transformation attached
vault write transform/role/logging-role transformations=ip-address



In [ ]:
# Mask a valid IP address
vault write transform/encode/logging-role value="10.100.1.87" \
    transformation=ip-address

# Tokenization Demo - Credit Card Number

The goal of Tokenization is to let end users' devices store the token rather than their sensitive values (such as credit card numbers) and still participate in transactions where the token is a stand-in for the sensitive value. For this reason the token Vault generates is completely unrelated (e.g. irreversible) to the sensitive value.

Furthermore, the Tokenization transform is designed to resist a number of attacks on the values produced during encode. In particular it is designed so that attackers cannot recover plaintext even if they steal the tokenization values from Vault itself. In the default mapping mode, even stealing the underlying transform key does not allow them to recover the plaintext without also possessing the encoded token. An attacker must have gotten access to all values in the construct.

## External Storage

Currently the PostgreSQL, MySQL, and MSSQL relational databases are supported as external storage backends for tokenization. The Schema Endpoint may be used to initialize and upgrade the necessary database tables. Vault uses a schema versioning table to determine if it needs to create or modify the tables when using that endpoint. If you make changes to those tables yourself, the automatic schema management may become out of sync and may fail in the future.

External stores may often be preferred due to their ability to achieve a much higher scale of performance, especially when used with batch operations.

https://developer.hashicorp.com/vault/docs/secrets/transform/tokenization#external-sql-stores 

In [ ]:
# Run a PostgreSQL database for tokenization demo
docker run --name postgres \
     -p 5432:5432 \
     --rm \
     -e POSTGRES_USER=root \
     -e POSTGRES_PASSWORD=mypassword \
     -d postgres

In [ ]:
# Verify Postgres database is running
docker ps

In [ ]:
# Create a role named "mobile-pay" with a transformation named "credit-card"
vault write transform/role/mobile-pay transformations=credit-card

In [ ]:
# Assumes your Vault is running on docker and connecting via the docker bridge network
# Obtain IP address of the postgres database for configuration
export POSTGRES_DB_IP=$(docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' postgres)
echo -e "Postgres IP Address is: $POSTGRES_DB_IP\n"
# Create a store which points to the postgres database
vault write transform/stores/postgres \
   type=sql \
   driver=postgres \
   supported_transformations=tokenization \
   connection_string="postgresql://{{username}}:{{password}}@$POSTGRES_DB_IP/root?sslmode=disable" \
   username=root \
   password=mypassword


In [ ]:
# Show that there is a new postgres store created
vault list transform/stores
echo
vault read transform/stores/postgres

In [ ]:
# Create a schema in postgres to store tokeniation artifacts
vault write transform/stores/postgres/schema transformation_type=tokenization \
    username=root password=mypassword

In [ ]:
# Create a transformation named "credit-card" which sets the generated token's max time-to-live (TTL) to 24 hours.
# Specify the postgres store
vault write transform/transformations/tokenization/credit-card \
  allowed_roles=mobile-pay \
  stores=postgres \
  max_ttl=24h


In [ ]:
# You can read back the properties of the transformation
vault read transform/transformations/tokenization/credit-card

In [ ]:
# Tokenize a credit card number.  Specify token time to live as 30 secs.
# Note policy to use tokenization - https://developer.hashicorp.com/vault/tutorials/adp/tokenization#tokenize-secrets

# vault write transform/encode/mobile-pay value=1111-2222-3333-4444 \
#      transformation=credit-card \
#      ttl=30s \
#      metadata="Organization=HashiCorp" \
#      metadata="Purpose=Travel" \
#      metadata="Type=AMEX"

export TOKEN_ENCODED_VALUE=$(vault write -format json transform/encode/mobile-pay value=1111-2222-3333-4444 \
     transformation=credit-card \
     ttl=30s \
     metadata="Organization=HashiCorp" \
     metadata="Purpose=Travel" \
     metadata="Type=AMEX" \
     | jq -r .data.encoded_value)
echo "encoded_value is : $TOKEN_ENCODED_VALUE"

# Note you can show the operation below within the 30s window and see the behaviour.

In [ ]:
# Retrieve metadata of the token
vault write transform/metadata/mobile-pay value=$TOKEN_ENCODED_VALUE transformation=credit-card

In [ ]:
# Validate the token value
vault write transform/validate/mobile-pay value=$TOKEN_ENCODED_VALUE transformation=credit-card

In [ ]:
# Validate if the credit card number has been tokenized already
vault write transform/tokenized/mobile-pay value=1111-2222-3333-4444 transformation=credit-card

In [ ]:
# Detokenize to retrieve the original plaintext credit card value

# vault write transform/decode/mobile-pay transformation=credit-card value=$TOKEN_ENCODED_VALUE

vault write -format json transform/decode/mobile-pay transformation=credit-card value=$TOKEN_ENCODED_VALUE | jq -r .data.decoded_value

## View Contents of PostgreSQL Database

In [ ]:
# Setup psql alias to the container
alias psql="docker exec -it postgres psql -U root"

# View tokens in the postgres store
psql -c '\x auto;' -c 'select * from tokens;'

## Convergent Tokenization

By default, tokenization produces a unique token for every encode operation. This makes the resulting token fully independent of its plaintext and expiration. Sometimes, though, it may be beneficial if the tokenization of a plaintext/expiration pair tokenizes consistently to the same value. For example if one wants to do a statistical analysis of the tokens as they relate to some other field in a database (without decoding the token), or if one needed to tokenize in two different systems but be able relate the results. In this case, one can create a tokenization transformation that is convergent.

When enabled at transformation creation time, Vault alters the calculation so that encoding a plaintext and expiration tokenizes to the same value every time, and storage keeps only a single entry of that token. Like the exportable mapping mode, convergence should only be enabled if needed. Convergent tokenization has a small performance penalty in external stores and a larger one in the built in store due to the need to avoid duplicate entries and to update metadata when convergently encoding. It is recommended that if one has some use cases that require convergence and some that do not, one should create two different tokenization transforms with convergence enabled on only one.

https://developer.hashicorp.com/vault/docs/secrets/transform/tokenization#convergence

In [ ]:
# Update the mobile-pay role with a convergent transformation named credit-card-convergent
vault write transform/role/mobile-pay transformations="credit-card, credit-card-convergent"

In [ ]:
# Create a transformation named credit-card-convergent which sets the enables the convergent encryption. 
# When you define a transformation, set convergent=true.
vault write transform/transformations/tokenization/credit-card-convergent \
     allowed_roles="*" \
     convergent=true

In [ ]:
# Encode a value using the credit-card-convergent transformation
vault write transform/encode/mobile-pay value=5555-6666-7777-8888 \
     transformation=credit-card-convergent

In [ ]:
# Run the command again
# The same encrypted value will be returned
vault write transform/encode/mobile-pay value=5555-6666-7777-8888 \
     transformation=credit-card-convergent

# Cleanup

In [ ]:
# Cleanup
# stop PostgreSQL container
docker stop postgres

# Disable transit engine
vault secrets disable transit

# Remove asymmetric encryption demo files
rm demo-key-rsa.pub
rm secret.txt
rm secret.enc

# Remove signing/verification demo files
rm message.txt
rm demo-sign-key-rsa-private.pem
rm demo-sign-key-rsa-public.pem
rm message.signature

# Disable transform engine
vault secrets disable transform

# Stop Vault container
docker stop vault-enterprise

# Other Useful Commands and Examples

In [ ]:
# Generate a string of random bytes
# https://developer.hashicorp.com/vault/api-docs/system/tools#generate-random-bytes
# e.g. AES 256 is 256 bits or 32 bytes
# Format can be base64 or hex
vault write sys/tools/random/32 format=base64
echo
vault write sys/tools/random/32 format=hex
echo 
vault write -field=random_bytes sys/tools/random/32 format=base64
echo
vault write -field=random_bytes sys/tools/random/32 format=hex

In [ ]:
# Example of generating a random 256-bit key (32 bytes) and storing it as a static secret in the KV engine
# Variable for my secret path
MOUNTPATH=secret
KEYNAME=demo-app-key-1
VALUENAME=value
# Generates a 256-bit key in base64 and stores it in the secret path with the name "value"
vault kv put $MOUNTPATH/$KEYNAME $VALUENAME="$(vault write -field=random_bytes sys/tools/random/32 format=base64)"
echo
# Application can read back key value with the proper permission policy.
vault kv get -mount=$MOUNTPATH -field $VALUENAME $KEYNAME

In [ ]:
# Demo of -output-policy flag
vault write -output-policy sys/tools/random/32 format=base64